In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from datetime import datetime
%matplotlib inline

In [2]:
# Hyperparameter
num_epochs = 10
batch_size = 32
learning_rate = 0.001
dropout_rate = 0.5
input_shape = (32, 32, 3)
num_classes = 10

In [3]:
# Build Model
inputs = layers.Input(input_shape)
l = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
l = layers.Activation('relu')(l)
l = layers.Conv2D(32, (3, 3), padding='SAME')(l)
l = layers.Activation('relu')(l)
l = layers.MaxPool2D(pool_size=(2, 2))(l)
l = layers.Dropout(dropout_rate)(l)

l = layers.Conv2D(64, (3, 3), padding='SAME')(l)
l = layers.Activation('relu')(l)
l = layers.Conv2D(64, (3, 3), padding='SAME')(l)
l = layers.Activation('relu')(l)
l = layers.MaxPool2D(pool_size=(2, 2))(l)
l = layers.Dropout(dropout_rate)(l)

l = layers.Flatten()(l)
l = layers.Dense(512)(l)
l = layers.Activation('relu')(l)
l = layers.Dense(128)(l)
l = layers.Activation('relu')(l)
l = layers.Dropout(dropout_rate)(l)
l = layers.Dense(num_classes)(l)
outputs = layers.Activation('softmax')(l)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='cnnv1')

In [29]:
# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=tf.keras.metrics.SparseCategoricalAccuracy()
)

In [30]:
# Preprocess Dataset
train_dataset_path = glob.glob('dataset/cifar/train/*.png')[:1000]
test_dataset_path = glob.glob('dataset/cifar/test/*.png')[:1000]

def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

class_category = [get_class_name(path) for path in train_dataset_path]
class_category = np.unique(class_category)

def get_sparse_label(path):
    global class_category
    fname = tf.strings.split(path, '_')[-1]
    cls_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(class_category == cls_name, tf.uint8)
    return tf.argmax(onehot)

def read_dataset(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.
    label = get_sparse_label(path)
    return image, label

def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset_path)
train_dataset = train_dataset.map(read_dataset, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(len(train_dataset_path))
train_dataset = train_dataset.repeat()

test_dataset = tf.data.Dataset.from_tensor_slices(train_dataset_path)
test_dataset = test_dataset.map(read_dataset, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

In [31]:
# Callbacks
log_dir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    write_graph=True,
    write_images=True,
    histogram_freq=1,
)

In [32]:
# Expert

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    predictions = model(images)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

In [33]:
log_dir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))
file_writer = tf.summary.create_file_writer(log_dir)

In [34]:
for epoch in range(num_epochs):
    
    for step, (images, labels) in enumerate(train_dataset):
        train_step(images, labels)
    
    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step+(step*epoch))
        tf.summary.scalar('train_loss', train_loss.result(), step=step+(step*epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=step+(step*epoch))
    
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)
    
    msg = 'Epoch {}, Loss {}, Accuracy {}, Test loss {}, Test accuracy {}'
    print(msg.format(
        epoch+1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100
    ))

KeyboardInterrupt: 